# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [67]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [68]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [69]:
spaceship.shape

(8693, 14)

**Check for data types**

**Check for missing values**

In [70]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [77]:
spaceship.dropna(inplace=True)
spaceship.shape

(6606, 12)

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [72]:
spaceship['Cabin'] = spaceship['Cabin'].str[0]

- Drop PassengerId and Name

In [76]:
spaceship.drop(['PassengerId','Name'], axis=1, inplace=True)

KeyError: "['PassengerId', 'Name'] not found in axis"

- For non-numerical columns, do dummies.

In [41]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [92]:
from sklearn.preprocessing import OneHotEncoder

# Subset data into numerical and categorical
spaceship_cat = spaceship.drop(columns=['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Transported'])
spaceship_num = spaceship.drop(columns=['HomePlanet','CryoSleep','Cabin','Destination','VIP'])

# One Hot Encodes the categorical variables
# Drop if binary drops the first value if there's only two values, otherwise keeps them all - yields .7662 predictive strength
# Drop first just drops first of each categorical - yields .7677 predictive strength
encoder = OneHotEncoder(drop= 'if_binary').fit(spaceship_cat)

cols = encoder.get_feature_names_out(input_features=spaceship_cat.columns)

spaceship_encode = pd.DataFrame(encoder.transform(spaceship_cat).toarray(),columns=cols)

spaceship_encode.reset_index(drop=True, inplace=True)
spaceship_num.reset_index(drop=True, inplace=True)

spaceship_prepped = pd.concat([spaceship_encode,spaceship_num],axis=1)

spaceship_prepped

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,...,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,False
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,24.0,109.0,9.0,25.0,549.0,44.0,True
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,58.0,43.0,3576.0,0.0,6715.0,49.0,False
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,33.0,0.0,1283.0,371.0,3329.0,193.0,False
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,16.0,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6601,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,41.0,0.0,6819.0,0.0,1643.0,74.0,False
6602,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,False
6603,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,26.0,0.0,0.0,1872.0,1.0,0.0,True
6604,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,32.0,0.0,1049.0,0.0,353.0,3235.0,False


In [83]:
spaceship_prepped.shape

(6606, 23)

**Perform Train Test Split**

In [93]:
features = spaceship_prepped.drop(columns = ['Transported'])
target = spaceship_prepped['Transported']

X_train, X_test, y_train, y_test = train_test_split(features,target, test_size=0.20,random_state=8675309)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [94]:
from sklearn.neighbors import KNeighborsClassifier
knnclass = knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

- Evaluate your model's performance. Comment it

In [95]:
knn.score(X_test, y_test)
#0.7662

0.7662632375189108